In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.2
3.2.1


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

In [5]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(1)

In [7]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [9]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [11]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x000001FCAFBB5EE0>)
ScalarOutcome('Total Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function sum_over at 0x000001FCAFBB5EE0>)
ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function=<function sum_over at 0x000001FCAFBB5EE0>)


In [13]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:31<00:00,  6.31it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [15]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['Expected Annual Damage', 'Total Investment Costs', 'Expected Number of Deaths'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,94,347.680127,1.5,0.310548,255.419031,1.5,0.137937,124.754514,1.0,0.933222,...,0,4,5,8,0,2,4,4,0,dikesnet
1,0,165.861605,1.5,0.367451,240.084105,1.5,0.685245,233.315337,1.0,0.393608,...,0,4,5,8,0,2,4,5,0,dikesnet
2,22,154.493628,10.0,0.957542,347.794423,1.0,0.934885,280.288026,10.0,0.550190,...,0,4,5,8,0,2,4,6,0,dikesnet
3,77,269.328875,1.0,0.245023,193.356869,10.0,0.859419,164.712478,1.0,0.355636,...,0,4,5,8,0,2,4,7,0,dikesnet
4,42,265.238400,1.0,0.578738,161.095839,1.0,0.670100,181.989953,10.0,0.788912,...,0,4,5,8,0,2,4,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,49,147.705652,1.0,0.898991,214.612626,1.0,0.379606,128.588198,10.0,0.623228,...,5,1,0,2,8,6,0,49,3,dikesnet
196,115,57.177594,1.0,0.635934,149.505188,1.5,0.630018,268.735407,1.0,0.751579,...,5,1,0,2,8,6,0,50,3,dikesnet
197,35,185.868758,1.5,0.096722,97.015117,1.5,0.352499,175.492869,1.0,0.951808,...,5,1,0,2,8,6,0,51,3,dikesnet
198,59,201.896185,10.0,0.056926,123.284631,1.0,0.565522,305.497458,10.0,0.022854,...,5,1,0,2,8,6,0,52,3,dikesnet


In [17]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,Expected Annual Damage,Total Investment Costs,Expected Number of Deaths
0,0.000000e+00,1.804887e+09,0.000000
1,5.592914e+05,1.804887e+09,0.000027
2,8.842154e+06,1.804888e+09,0.000695
3,7.727344e+06,1.804888e+09,0.000607
4,0.000000e+00,1.804887e+09,0.000000
...,...,...,...
195,2.003923e+07,2.221172e+09,0.022649
196,9.254285e+06,2.221172e+09,0.006610
197,5.017929e+07,2.221172e+09,0.034824
198,7.218554e+08,2.221172e+09,1.128550


In [19]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [21]:
lever_settings = {lever.name: 0 for lever in levers}
Base_Case = Policy("Zero Policy", **lever_settings)

In [23]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 40000
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies = [Base_Case])

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 40000 scenarios * 1 policies * 1 model(s) = 40000 experiments
100%|██████████████████████████████████| 40000/40000 [2:12:39<00:00,  5.03it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [29]:
experiments, outcomes = results

In [27]:
from ema_workbench import save_results
save_results((experiments, outcomes), './data/results.tar.gz')

[MainProcess/INFO] results saved successfully to C:\Users\maxva\Downloads\epa141A_open-master\epa141A_open-master\final assignment\data\results.tar.gz


In [31]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,Expected Annual Damage,Total Investment Costs,Expected Number of Deaths
0,3.016100e+09,0.0,1.972177
1,1.673129e+09,0.0,1.084666
2,3.159433e+09,0.0,1.951017
3,3.311397e+08,0.0,0.572630
4,4.766298e+08,0.0,0.365391
...,...,...,...
39995,2.166199e+09,0.0,2.044284
39996,2.745800e+09,0.0,2.187244
39997,6.464982e+08,0.0,0.636409
39998,2.915529e+09,0.0,2.002077
